In [3]:
import os
from dateutil.parser import parse as dateutil_parse
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler as skStandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.cross_validation import train_test_split,cross_val_score

PATH = 'season_1/'
CLEAN_PATH = PATH+'clean/'

%matplotlib inline

In [4]:
class myStandardScaler(skStandardScaler):
    '''
    Only use fit_transform and tranform, specific for dealing with pd.DataFrame.
    Only scale the numerical features.
    '''

    def fit_transform(self, X):
        Xnumerical = X[X.columns[X.dtypes != bool]]
        Xdummy = X[X.columns[X.dtypes == bool]]
        scaledXnumerical = super(myStandardScaler, self).fit_transform(Xnumerical)
        Xnumerical = pd.DataFrame(scaledXnumerical, index=Xnumerical.index, columns=Xnumerical.columns)
        return pd.concat([Xnumerical, Xdummy], axis=1)

    def transform(self, X):
        Xnumerical = X[X.columns[X.dtypes != bool]]
        Xdummy = X[X.columns[X.dtypes == bool]]
        scaledXnumerical = super(myStandardScaler, self).transform(Xnumerical)
        Xnumerical = pd.DataFrame(scaledXnumerical, index=Xnumerical.index, columns=Xnumerical.columns)
        return pd.concat([Xnumerical, Xdummy], axis=1)

In [5]:
cluster_map = pd.read_csv(CLEAN_PATH+'cluster_map.csv',index_col=0)
poi = pd.read_csv(CLEAN_PATH+'poi.csv',index_col=0)
train_order = pd.read_pickle(CLEAN_PATH+'train_order.pickle')
test_order = pd.read_pickle(CLEAN_PATH+'test_order.pickle')
train_traffic = pd.read_pickle(CLEAN_PATH+'train_traffic.pickle')
test_traffic = pd.read_pickle(CLEAN_PATH+'test_traffic.pickle')
train_weather = pd.read_pickle(CLEAN_PATH+'train_weather.pickle')
test_weather = pd.read_pickle(CLEAN_PATH+'test_weather.pickle')
test_target = pd.read_csv(CLEAN_PATH+'test_target.csv',index_col=0,parse_dates=True)

In [6]:
train_order_index = pd.Series(range(len(train_order)),index=train_order['Time'])
test_order_index = pd.Series(range(len(test_order)),index=test_order['Time'])
train_traffic_index = pd.Series(range(len(train_traffic)),index=train_traffic['datetime'])
test_traffic_index = pd.Series(range(len(test_traffic)),index=test_traffic['datetime'])

In [7]:
def process_order(order):
    def pclass(p):
        class_set = range(5,20,5)+range(20,100,10)+range(100,501,100)
        idx_set = [p>cls for cls in class_set]
        return idx_set.index(False) if sum(idx_set)!=len(idx_set) else len(idx_set)
    order['timeslot'] = order['Time'].map(lambda x: (x.hour*60+x.minute)/10+1)
    order['datetimeslot'] = order['Time'].map(lambda x: x.year*10000+x.month*100+x.day)*1000+order['timeslot']
    order['pclass'] = order['Price'].map(lambda x: pclass(x))
    order = pd.concat([order,pd.get_dummies(order['pclass'],'pclass').applymap(lambda x: {1.0: True, 0.0: False}[x])],axis=1)
    return order

In [8]:
train_order = process_order(train_order)
test_order = process_order(test_order)

In [9]:
def map_group(group):
    res = pd.Series()
    res['request'] = group['request'].count()
    res['answer'] = group['request'].sum()
    res['price_avg'] = group['Price'].mean()
    pclass_cols = filter(lambda x: x[:7]=='pclass_',group.columns)
    pclass_values = group[pclass_cols].sum()
    res = pd.concat([res,pclass_values])
    return res

In [10]:
districts = train_order['start_district_hash'].unique()

In [11]:
set(districts) == set(test_order['start_district_hash'].unique())

True

In [12]:
def get_order_group(order):
    order_group = dict()
    for district in districts:
        tmp = order[order['start_district_hash']==district]
        order_group[district] = tmp.groupby('datetimeslot').apply(lambda g: map_group(g))      
    return order_group

In [13]:
train_order_group = get_order_group(train_order)

In [14]:
test_order_group = get_order_group(test_order)

In [37]:
test_order_group['d5cb17978de290c56e84c9cf97e63186']

,request,answer,price_avg,pclass_0,pclass_1,pclass_2,pclass_3,pclass_4,pclass_5,pclass_6,pclass_7,pclass_8,pclass_9,pclass_10,pclass_11,pclass_12,pclass_13,pclass_14,pclass_15,pclass_16
datetimeslot,,,,,,,,,,,,,,,,,,,,
20160122045,1.0,1.0,37.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122057,2.0,2.0,13.500000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122080,1.0,0.0,27.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122092,1.0,0.0,28.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122093,1.0,1.0,26.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122103,1.0,1.0,28.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122104,1.0,1.0,31.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122117,7.0,0.0,59.428571,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160122127,2.0,2.0,21.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
def train_test_order(order, dts):
    '''
    This funciton will not consider the first 3 time slot, for all the days. 
    It could be changed when test data set changes.
    '''
    timeslot = pd.get_dummies(dts.map(lambda x: int(x%1000)),'timeslot')
    #     target_timeslot = ['timeslot_'+str(i+1) for i in range(144)]
    target_timeslot = ['timeslot_'+str(i+1) for i in range(144)]
    timeslot = pd.DataFrame(timeslot, columns=target_timeslot).fillna(0)
    timeslot.index = dts
    timeslot = timeslot.applymap(lambda x: {1.0: True, 0.0: False}[x])
    train = pd.concat([order.ix[dts-1].rename(columns=lambda x: '1_'+x,index=lambda x: x+1),
                       order.ix[dts-2].rename(columns=lambda x: '2_'+x,index=lambda x: x+2),
                       order.ix[dts-3].rename(columns=lambda x: '3_'+x,index=lambda x: x+3),
                       timeslot], axis=1)
    train = train.fillna(0)
    test = (order['request'].ix[train.index]-order['answer'].ix[train.index])
    return [train,test]

In [47]:
train_XYgroup = dict()
for district in train_order_group:
    train_XYgroup[district] = train_test_order(train_order_group[district],train_order_group[district].index)

In [48]:
train_XYgroup['d5cb17978de290c56e84c9cf97e63186'][0]

,1_request,1_answer,1_price_avg,1_pclass_0,1_pclass_1,1_pclass_2,1_pclass_3,1_pclass_4,1_pclass_5,1_pclass_6,...,timeslot_135,timeslot_136,timeslot_137,timeslot_138,timeslot_139,timeslot_140,timeslot_141,timeslot_142,timeslot_143,timeslot_144
datetimeslot,,,,,,,,,,,,,,,,,,,,,
20160101005,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101011,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101045,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101050,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101052,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101053,1.0,1.0,32.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101057,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101060,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160101061,3.0,2.0,9.000000,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [49]:
test_XYgroup = dict()
for district in test_order_group:
    test_XYgroup[district] = train_test_order(test_order_group[district],test_target['datetimeslot'])

In [50]:
test_XYgroup['d5cb17978de290c56e84c9cf97e63186'][0]

,1_request,1_answer,1_price_avg,1_pclass_0,1_pclass_1,1_pclass_2,1_pclass_3,1_pclass_4,1_pclass_5,1_pclass_6,...,timeslot_135,timeslot_136,timeslot_137,timeslot_138,timeslot_139,timeslot_140,timeslot_141,timeslot_142,timeslot_143,timeslot_144
datetimeslot,,,,,,,,,,,,,,,,,,,,,
20160122046,1.0,1.0,37.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122058,2.0,2.0,13.500000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122070,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122082,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122094,1.0,1.0,26.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122106,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122118,7.0,0.0,59.428571,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122130,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
20160122142,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,False


In [51]:
for district in train_order_group:
    sample_scaler = myStandardScaler()
    train_XYgroup[district][0] = sample_scaler.fit_transform(train_XYgroup[district][0])
    test_XYgroup[district][0] = sample_scaler.transform(test_XYgroup[district][0])

In [52]:
rfr_models = {district: RandomForestRegressor(100, max_depth=12) for district in districts}

In [53]:
def evaluate(target, predict):
    return ((target-predict).abs()/target).replace(np.inf,0).mean()

In [54]:
test_XYgroup['d5cb17978de290c56e84c9cf97e63186'][0]

,1_request,1_answer,1_price_avg,1_pclass_0,1_pclass_1,1_pclass_2,1_pclass_3,1_pclass_4,1_pclass_5,1_pclass_6,...,timeslot_135,timeslot_136,timeslot_137,timeslot_138,timeslot_139,timeslot_140,timeslot_141,timeslot_142,timeslot_143,timeslot_144
datetimeslot,,,,,,,,,,,,,,,,,,,,,
20160122046,-0.071024,0.561767,0.830725,-0.146581,-0.215284,-0.206726,-0.246110,-0.391793,1.585448,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122058,0.426906,1.661684,-0.090129,-0.146581,3.478544,-0.206726,2.261137,-0.391793,-0.280458,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122070,-0.568953,-0.538149,-0.619130,-0.146581,-0.215284,-0.206726,-0.246110,-0.391793,-0.280458,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122082,-0.568953,-0.538149,-0.619130,-0.146581,-0.215284,-0.206726,-0.246110,-0.391793,-0.280458,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122094,-0.071024,0.561767,0.399687,-0.146581,-0.215284,-0.206726,-0.246110,0.813155,-0.280458,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122106,-0.568953,-0.538149,-0.619130,-0.146581,-0.215284,-0.206726,-0.246110,-0.391793,-0.280458,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122118,2.916553,-0.538149,1.709595,-0.146581,-0.215284,-0.206726,-0.246110,-0.391793,1.585448,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122130,-0.568953,-0.538149,-0.619130,-0.146581,-0.215284,-0.206726,-0.246110,-0.391793,-0.280458,-0.193078,...,False,False,False,False,False,False,False,False,False,False
20160122142,-0.568953,-0.538149,-0.619130,-0.146581,-0.215284,-0.206726,-0.246110,-0.391793,-0.280458,-0.193078,...,False,False,False,False,False,False,False,True,False,False


In [55]:
test_prediction = dict()
for district, model in rfr_models.items():
    model.fit(*train_XYgroup[district])
    train_prediction = pd.Series(np.floor(model.predict(train_XYgroup[district][0])),index=train_XYgroup[district][1].index)
    test_prediction[district] = np.floor(model.predict(test_XYgroup[district][0])) - test_XYgroup[district][1].fillna(0)
    print district,'r2 score', model.score(*train_XYgroup[district])
    print district,'metric', evaluate(model.predict(train_XYgroup[district][0]),train_prediction)

38d5ad2d22b61109fd8e7b43cd0e8901 r2 score 0.977796483509
38d5ad2d22b61109fd8e7b43cd0e8901 metric 0.168790517239
08f5b445ec6b29deba62e6fd8b0325a6 r2 score 0.694534114285
08f5b445ec6b29deba62e6fd8b0325a6 metric 0.836690823685
364bf755f9b270f0f9141d1a61de43ee r2 score 0.865076863328
364bf755f9b270f0f9141d1a61de43ee metric 0.388533021496
49ac89aa860c27e26c0836cb8dab2df2 r2 score 0.43617151884
49ac89aa860c27e26c0836cb8dab2df2 metric 0.889708827166
8bb37d24db1ad665e706c2655d9c4c72 r2 score 0.712412339976
8bb37d24db1ad665e706c2655d9c4c72 metric 0.485754112359
08232402614a9b48895cc3d0aeb0e9f2 r2 score 0.450615366107
08232402614a9b48895cc3d0aeb0e9f2 metric 0.934004376407
b702e920dcd2765e624dc1ce3a770512 r2 score 0.808462838004
b702e920dcd2765e624dc1ce3a770512 metric 0.295312925757
52d7b69796362a8ed1691a6cc02ddde4 r2 score 0.443850937279
52d7b69796362a8ed1691a6cc02ddde4 metric 0.916399471539
62afaf3288e236b389af9cfdc5206415 r2 score 0.967073784771
62afaf3288e236b389af9cfdc5206415 metric 0.058113

In [56]:
train_order_group['7f84bdfc2b6d4541e1f6c0a3349e0251']

,request,answer,price_avg,pclass_0,pclass_1,pclass_2,pclass_3,pclass_4,pclass_5,pclass_6,pclass_7,pclass_8,pclass_9,pclass_10,pclass_11,pclass_12,pclass_13,pclass_14,pclass_15,pclass_16
datetimeslot,,,,,,,,,,,,,,,,,,,,
20160101002,1.0,1.0,40.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101041,2.0,1.0,21.500000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101043,1.0,1.0,96.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
20160101044,1.0,1.0,7.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101045,1.0,1.0,20.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101046,4.0,2.0,16.750000,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101049,2.0,0.0,59.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101050,3.0,3.0,13.000000,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101051,1.0,1.0,12.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# pd.DataFrame({'prediction':test_prediction['d524868ce69cb9db10fc5af177fb9423'],'district':1})

In [59]:
prediction2submit(test_prediction).to_csv(PATH+'submit/order_rf.csv',header=False,index=None)

In [60]:
prediction2submit(test_prediction)[prediction2submit(test_prediction)['district']==15]

,district,dts,prediction
datetimeslot,,,
20160122046,15,2016-01-22-46,0.0
20160122058,15,2016-01-22-58,0.0
20160122070,15,2016-01-22-70,0.0
20160122082,15,2016-01-22-82,0.0
20160122094,15,2016-01-22-94,0.0
20160122106,15,2016-01-22-106,0.0
20160122118,15,2016-01-22-118,1.0
20160122130,15,2016-01-22-130,0.0
20160122142,15,2016-01-22-142,0.0


In [61]:
cluster_map.ix['3a43dcdff3c0b66b1acb1644ff055f9d']
cluster_map[cluster_map['district_id']==15]

,district_id
start_district_hash,
d5cb17978de290c56e84c9cf97e63186,15


In [63]:
train_order_group['d5cb17978de290c56e84c9cf97e63186']

,request,answer,price_avg,pclass_0,pclass_1,pclass_2,pclass_3,pclass_4,pclass_5,pclass_6,pclass_7,pclass_8,pclass_9,pclass_10,pclass_11,pclass_12,pclass_13,pclass_14,pclass_15,pclass_16
datetimeslot,,,,,,,,,,,,,,,,,,,,
20160101005,1.0,1.0,50.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101011,1.0,0.0,35.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101045,1.0,0.0,65.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101050,1.0,1.0,30.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101052,1.0,1.0,32.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101053,1.0,1.0,5.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101057,2.0,2.0,23.000000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101060,3.0,2.0,9.000000,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20160101061,1.0,1.0,51.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
